# Placo Arm Control

This is a test-purpose control library for walli arm. Instead of directly solving an iterative IK, it formualtes a QP program with multiple targets and constraints, then solves it rapidly with an off-the-shelf solver (eiquadprog).

The output of this program is a series of joint position commands, published through a ROS2 publisher in the format of MultiDOFCommands. You may bring up any simulator (e.g. Gazebo with ros2_control configured, or Isaac Sim with the arm imported and omnigraph setted up) to see the robot's end effector in movement. It should draw a "inf" sign in the air.

In [72]:
# install requirements
!pip install placo

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [83]:
import placo
import time
import numpy as np
from placo_utils.visualization import robot_frame_viz, robot_viz, frame_viz, point_viz

robot = placo.RobotWrapper(
    "/home/keseterg/Documents/RoboJackets/urc_ws/src/urc-software/urc_arm_models/urdf/z1_description/z1.urdf",
    placo.Flags.ignore_collisions
)
robot.state.q = robot.neutral_state().q
viz = robot_viz(robot)

In [84]:
def translation(arr):
    return arr[0:3, 3]

In [85]:
solver = placo.KinematicsSolver(robot)
solver.mask_fbase(True)

pEE = translation(robot.get_T_world_frame("link06"))
ee_frame_task = solver.add_position_task("link06", pEE)
ee_frame_task.configure("frame::ee", "soft", 10.0)

In [120]:
point_viz("target", pEE)
ee_frame_task.target_world = pEE.copy()
solver.dt = 0.01
robot.update_kinematics()
solver.solve(True)
solver.dump_status()
robot_frame_viz(robot, "link06")
viz.display(robot.state.q)

* Kinematics Tasks:
  * frame::ee [position]
    - Priority: soft (weight:10)
    - Error: 0.000000 [m]



In [115]:
pEE[2] += 0.1